In [1]:
import torch
import datasets
import os
import time
import numpy as np
import sys
import pickle
import torch.nn.functional as F
from transformers import (
    LlamaForCausalLM,
    GenerationConfig,
    T5ForConditionalGeneration,
    AutoTokenizer,
    MT5ForConditionalGeneration,
    AutoModel,
    Conversation,
    ConversationalPipeline,
    T5Tokenizer,
    BertForSequenceClassification,
    BertModel,
    BertConfig,
    TextClassificationPipeline,
    AutoTokenizer
)
from typing import Any, Dict, List, Optional
from transformers import pipeline

In [2]:
import logging
logger = logging.getLogger("test")

### Skip this

In [2]:
llama_causal = LlamaForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
from hqq.models.hf.base import AutoHQQHFModel
from hqq.core.quantize import BaseQuantizeConfig
quant_config = BaseQuantizeConfig(nbits=4, group_size=64)
AutoHQQHFModel.quantize_model(llama_causal, quant_config=quant_config, 
                                    compute_dtype=torch.float16, 
                                    device="cuda")
# 1.4gb

100%|██████████| 155/155 [00:17<00:00,  9.00it/s]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): HQQLinear(in_features=2048, out_features=2048, bias=False)
          (k_proj): HQQLinear(in_features=2048, out_features=256, bias=False)
          (v_proj): HQQLinear(in_features=2048, out_features=256, bias=False)
          (o_proj): HQQLinear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): HQQLinear(in_features=2048, out_features=5632, bias=False)
          (up_proj): HQQLinear(in_features=2048, out_features=5632, bias=False)
          (down_proj): HQQLinear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()


In [7]:
from hqq.models.hf.base import AutoHQQHFModel
AutoHQQHFModel.save_quantized(llama_causal, "./tinyllama_hqq_4_64")

### Continue

In [3]:
MODEL = "shahules786/Safetybot-t5-base"
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda:0


In [4]:
#@title
class SafetyTokenizer(T5Tokenizer):

    def _build_conversation_input_ids(self, conversation: "Conversation") -> List[int]:
        inputs = []
        for is_user, text in conversation.iter_texts():
            if is_user:
                # We need to space prefix as it's being done within blenderbot
                inputs.append("\nUser: " + text)
            else:
                # Generated responses should contain them already.
                inputs.append("\nbot: " + text)

        user_input = ":".join(inputs.pop(-1).split(":")[1:])
        context = self.sep_token.join(inputs)

        input_tokens = self.encode(user_input, add_special_tokens=False)
        max_len = self.model_max_length - (len(input_tokens) + 2)
        context = self.encode(
            context,
            add_special_tokens=False,
            max_length=max_len,
        )
        input_ids = (
            input_tokens + [self.context_token_id] + context + [self.eos_token_id]
        )
        input_ids = input_ids + [self.pad_token_id] * max(
            0, (self.model_max_length - len(input_ids))
        )
        mask = [1] * len(input_ids) + [self.pad_token_id] * (
            self.model_max_length - len(input_ids)
        )
        if len(input_ids) > self.model_max_length:
            input_ids = input_ids[-self.model_max_length :]
            mask = mask[-self.model_max_length :]
            logger.warning(
                f"Trimmed input from conversation as it was longer than {self.model_max_length} tokens."
            )
        return input_ids, mask


class SafetyPipeline(ConversationalPipeline):
    def preprocess(
        self, conversation: Conversation, min_length_for_response=32
    ) -> Dict[str, Any]:
        if not isinstance(conversation, Conversation):
            raise ValueError("ConversationalPipeline, expects Conversation as inputs")
        if conversation.new_user_input is None:
            raise ValueError(
                f"Conversation with UUID {type(conversation.uuid)} does not contain new user input to process. "
                "Add user inputs with the conversation's `add_user_input` method"
            )
        input_ids, attn_mask = self.tokenizer._build_conversation_input_ids(
            conversation
        )

        input_ids = torch.tensor([input_ids])
        attn_mask = torch.tensor([attn_mask])

        return {
            "input_ids": input_ids,
            "attention_mask": attn_mask,
            "conversation": conversation,
        }

    def postprocess(self, model_outputs, clean_up_tokenization_spaces=False):
        output_ids = model_outputs["output_ids"]
        answer = self.tokenizer.decode(
            output_ids[0],
            skip_special_tokens=False,
            clean_up_tokenization_spaces=clean_up_tokenization_spaces,
        )
        return answer

SPECIAL_TOKENS = {"context_token":"<ctx>","sep_token":"<sep>","label_token":"<cls>","rot_token":"<rot>"}
# load_safety model into gpu
def load_model(model_name):

    if "mt5" in model_name:
        model = MT5ForConditionalGeneration.from_pretrained(model_name)
    else:
        model = T5ForConditionalGeneration.from_pretrained(model_name)

    tokenizer = SafetyTokenizer.from_pretrained(
        MODEL, padding_side="right", truncation_side="right", model_max_length=256
    )

    # add SPECIAL_TOKENS
    for key,value in SPECIAL_TOKENS.items():
        setattr(tokenizer,key,value)
        tokenizer.add_tokens([value])
        setattr(tokenizer,key+"_id",tokenizer.encode(value)[0])

    model.resize_token_embeddings(len(tokenizer))

    # init model max_length for t5
    model.config.max_length = 512

    model.eval()

    return model, tokenizer

In [5]:
model, tokenizer = load_model(MODEL)
safety_bot = SafetyPipeline(model=model,tokenizer=tokenizer,device=device)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'SafetyTokenizer'.
You are using the default legacy behaviour of the <class '__main__.SafetyTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
def get_safety_models_opinion(user_prompt, conversation=None):
    if not conversation:
        conversation = Conversation(user_prompt)
        resp = safety_bot(conversation)
        return resp, conversation
    conversation.add_user_input(user_prompt)
    resp = safety_bot(conversation)
    return resp, conversation

In [20]:
cls_bert_model = BertForSequenceClassification.from_pretrained("../artifacts/topic/")
bert_model_pipeline = pipeline("text-classification", model=cls_bert_model, tokenizer=AutoTokenizer.from_pretrained("bert-base-uncased"))

In [23]:
post = "I HATE ALL LESBIANS GAYS TRANSEXUALS PANSEXUALS THEY ALL SUCK SO MUCH AAAAAAAA goofy."
safety_bot_out = get_safety_models_opinion("""I am incredibly racists against bread loafs.""")
warning_level = safety_bot_out[0].split("<ctx>")[0].replace("<cls>", "").replace("_", " ").strip()
warning_description = safety_bot_out[0].split("<ctx>")[1].replace("</s>", "").strip()
topic_out = bert_model_pipeline("I HATE ALL LESBIANS GAYS TRANSEXUALS PANSEXUALS THEY ALL SUCK SO MUCH AAAAAAAA goofy.")
topic_name = topic_out[0]['label']
topic_confidence = topic_out[0]['score']

In [2]:
from hqq.models.hf.base import AutoHQQHFModel
llama_causal = AutoHQQHFModel.from_quantized("../artifacts/tinyllama_hqq_4_64")

100%|██████████| 155/155 [00:00<00:00, 7943.66it/s]


In [3]:
from transformers import AutoTokenizer
llama_tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

In [4]:
# . Here are some examples of valid responses.
#         Example 1: <user> I AM CRYING WAAAAAA!!!! I want ice cream :( <user><assistant> sad <assistant>
#         Example 2: <user> how could YOU BETRAY ME?!?!? <user><assistant> angry <assistant>
#         Example 3: <user> YAY FRIENDSHIP IS MAGIC!! <user><assistant> happy <assistant>
input_ids = llama_tokenizer.apply_chat_template([
    {
        "role": "system",
        "content": """You are now an expert on recognizing emotions in conversations. You understand emotions to all fall into one of the following categories: [angry, happy, sad, scared, other].
        """,
    },
    {
        "role": "user",
        "content": "Classify the following based on expert knowledge: I AM CRYING WAAAAAA!!!! I want ice cream :(",
    },
    {
        "role": "assistant",
        "content": "sad",
    },
    {
        "role": "user",
        "content": "Classify the following as before: how could YOU BETRAY ME?!?!?",
    },
    {
        "role": "assistant",
        "content": "angry",
    },
    {
        "role": "user",
        "content": "Classify the following as before: euphoria is filling my eeyes or something!",
    }
], return_tensors='pt')

In [188]:
# [ACTION] Moderation action to take [ACTION] 
# We now want individuals on our social media site who are suicidal to not 
# see each other's posts. Each person has made multiple social media 
# posts. Here are some statistics on each user's posts, formatted as a 
# list of [description of harmful content]#[topic of post]#[sentiment of 
# post]#[action taken] entries
# It's not socially acceptable to drown yourself#psychiatric_or_mental_illness#0.02#needs intervention
# Fantasizing about dying is not mentally healthy#psychiatric_or_mental_illness#0.01#needs intervention
# You should not think you are worthless#psychiatric_or_mental_illness#0.05#needs intervention
# Drugs are not a depression curer#drugs#0.02#needs intervention

input_ids = llama_tokenizer.apply_chat_template([
    {
        "role": "system",
        "content": """You are an expert on rating dangerous personalities.
        """,
    },
    # {
    #     "role": "user",
    #     "content": """Summarize the behaviour of an imaginary person's with their description and danger from 1 to 10.
    #     """,
    # },
    # {
    #     "role": "assistant",
    #     "content": "[DESCRIPTION] Andrew is a go lucky friendly individual who isn't afraid to voice their own opinions [DANGER] 3 [END]",
    # },
    # {
    #     "role": "user",
    #     "content": """Summarize the behaviour of a different imaginary person with the previous format.
    #     """,
    # },
    # {
    #     "role": "assistant",
    #     "content": "[DESCRIPTION] Sarah is a fierce and determined individual who is always looking for a challenge [DANGER] 1 [END]",
    # },
    {
        "role": "user",
        "content": """Felicia received these warnings for her social media posts. Summarize the behaviour of Felicia:
        It's wrong to think non-people are non-persons.
        It's not right to think all black people are criminals.
        """,
    },# [ACTIONS] All posts should be deleted
    {
        "role": "assistant",
        "content": "[DESCRIPTION] The user is a social pariah who hates black people. [DANGER] 9 [END]",
    }, 
    {
        "role": "user",
        "content": """Cody received these warnings for her social media posts. Summarize the behaviour of Cody:
        It's bad to be depressed.
        It's bad to use ropes that way.
        It's bad to think no one would care if you were gone
        """,
    },
    {
        "role": "assistant",
        "content": "[DESCRIPTION] The user is a depressive who uses ropes as a way to cope with their emotions. [DANGER] 7 [END]",
    },
    {
        "role": "user",
        "content": """Brock received these warnings for her social media posts. Summarize the behaviour of Brock:
        Alcohol does not cure depression.
        It's bad to think no one would care if you were gone
        You shouldn't promote alcoholism
        It's bad to be depressed.
        It's bad to use ropes that way.
        You shouldn't drive while drunk
        """,
    },
], return_tensors='pt')

In [ ]:
# [ACTION] Moderation action to take [ACTION]
input_ids = llama_tokenizer.apply_chat_template([
    {
        "role": "system",
        "content": """You are an expert on rating dangerous personalities.
        """,
    },
], return_tensors='pt')

In [189]:
gen_cfg = GenerationConfig.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
gen_cfg.max_length = 768

In [190]:
import textwrap
textwrap.wrap(llama_tokenizer.decode(llama_causal.generate(input_ids.cuda().long(), generation_config=gen_cfg)[0]))

['<|system|> You are an expert on rating dangerous personalities.',
 '</s>  <|user|> Felicia received these warnings for her social media',
 "posts. Summarize the behaviour of Felicia:         It's wrong to think",
 "non-people are non-persons.         It's not right to think all black",
 'people are criminals.         </s>  <|assistant|> [DESCRIPTION] The',
 'user is a social pariah who hates black people. [DANGER] 9 [END]</s>',
 '<|user|> Cody received these warnings for her social media posts.',
 "Summarize the behaviour of Cody:         It's bad to be depressed.",
 "It's bad to use ropes that way.         It's bad to think no one would",
 'care if you were gone         </s>  <|assistant|> [DESCRIPTION] The',
 'user is a depressive who uses ropes as a way to cope with their',
 'emotions. [DANGER] 7 [END]</s>  <|user|> Brock received these warnings',
 'for her social media posts. Summarize the behaviour of Brock:',
 "Alcohol does not cure depression.         It's bad to think no one"

In [ ]:
['<|system|> You are an expert on rating dangerous personalities.',
 "</s>  <|user|> Summarize the behaviour of an imaginary person's with",
 'their description and danger from 1 to 10.         </s>  <|assistant|>',
 "[DESCRIPTION] Andrew is a go lucky friendly individual who isn't",
 'afraid to voice their own opinions [DANGER] 3 [END]</s>  <|user|>',
 'Summarize the behaviour of a different imaginary person with the',
 'previous format.         </s>  <|assistant|> [DESCRIPTION] Sarah is a',
 'fierce and determined individual who is always looking for a challenge',
 '[DANGER] 1 [END]</s>  <|user|> Felicia received these warnings for her',
 "social media posts. Summarize the behaviour of Felicia:         It's",
 "wrong to think non-people are non-persons.         It's not right to",
 'think all black people are criminals.         </s>  <|assistant|>',
 '[DESCRIPTION] The user is a social pariah who hates black people.',
 '[DANGER] 9 [END]</s>  <|user|> Cody received these warnings for her',
 "social media posts. Summarize the behaviour of Cody:         It's bad",
 "to be depressed.         It's bad to use ropes that way.         It's",
 'bad to think no one would care if you were gone         </s>',
 '<|assistant|> [DESCRIPTION] The user is a depressive who uses ropes as',
 'a way to cope with their emotions. [DANGER] 7 [END]</s>  <|user|>',
 'Brock received these warnings for her social media posts. Summarize',
 'the behaviour of Brock:         Alcohol does not cure depression.',
 "It's bad to think no one would care if you were gone         You",
 "shouldn't promote alcoholism         It's bad to be depressed.",
 "It's bad to use ropes that way.         You shouldn't drive while",
 'drunk         </s>  <|assistant|> [DESCRIPTION] The user is a',
 'depressive who uses alcohol as a coping mechanism. [DANGER] 7',
 '[END]</s>']


    # {
    #     "role": "user",
    #     "content": """Alice received these warnings for her social media posts. Summarize the behaviour of Alice:
    #     Cannibalism is not legal
    #     It is wrong to kill people
    #     Individuals are not your playthings
    #     """,
    # },
    # {
    #     "role": "user",
    #     "content": """With the previous formatting, Describe a different person who has received these warnings from society: \"It's bad to be depressed.\" "Suicide" "negative"
    #     \"It's wrong to think non-people are non-persons.\" "Racism" "negative"
    #     \"It's not right to think all black people are criminals\" "Racism" "negative"
    #     """,
    # },
    # {
    #     "role": "assistant",
    #     "content": "[DANGER] 0.99 [DANGER] [DESCRIPTION] This bozo is a sociopath who believes that all people are inherently flawed and inferior. [DESCRIPTION] [ACTION] Ban this person [ACTION]",
    # },
    # {
    #     "role": "user",
    #     "content": "Describe a person's 5 personality traits based on expert knowledge with the previous list formatting",
    # },
    # {
    #     "role": "user",
    #     "content": """It's bad to be depressed.
    #     It's wrong to think non-people are non-persons.
    #     It's not right to think all black people are criminals
    #     """
    # }